In [2]:
from flowmason import conduct, MapReduceStep, SingletonStep, load_artifact # TODO: install this package from me: https://github.com/smfsamir/flowmason

# from allosaurus.app import read_recognizer
import panphon.distance # https://github.com/dmort27/panphon

# TODO: you'll have to install these packages. Let me know if there's any trouble here; you should be able to do `pip install` for all of them
import polars as pl
import torch
import soundfile as sf
import os
import ipdb
import zipfile
import os
from scipy.io import wavfile
import shutil
from praatio import textgrid as tgio

from typing import List
from collections import OrderedDict
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC 


HF_CACHE_DIR= r"C:\Users\david\OneDrive\Desktop\school\URO\URO_multilingual_speech_recognition\outputs\hf_cache_dir"
SCRATCH_DIR= r"C:\Users\david\OneDrive\Desktop\school\URO\URO_multilingual_speech_recognition\outputs\scratch_dir"

In [3]:
arpabet_to_ipa = {
    'AA': 'ɑ', 
    'AE': 'æ',
    'AH': 'ʌ',
    'AO': 'ɔ',
    'AW': 'aʊ',
    'AY': 'aɪ',
    'EH': 'ɛ',
    'ER': 'ɝ',
    'EY': 'eɪ',
    'IH': 'ɪ',
    'IY': 'i',
    'OW': 'oʊ',
    'OY': 'ɔɪ',
    'UH': 'ʊ',
    'UW': 'u',
    'B': 'b',
    'CH': 'tʃ',
    'D': 'd',
    'DH': 'ð',
    'F': 'f',
    'G': 'g',
    'HH': 'h',
    'JH': 'dʒ',
    'K': 'k',
    'L': 'l',
    'M': 'm',
    'N': 'n',
    'NG': 'ŋ',
    'P': 'p',
    'R': 'ɹ',
    'S': 's',
    'SH': 'ʃ',
    'T': 't',
    'TH': 'θ',
    'V': 'v',
    'W': 'w',
    'Y': 'j',
    'Z': 'z',
    'ZH': 'ʒ',
}
def get_data_iterator():
    iteratable = process_zip(r'C:\Users\david\OneDrive\Desktop\school\URO_ARTIC_DATASET\l2arctic_release_v5.0.zip', 1)
    return iteratable

def process_zip(zip_path, numberofspeaker = 1, numberofsampleperSpeaker = 1):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all the contents into a temporary directory
        temp_dir = 'temp_extracted'
        zip_ref.extractall(temp_dir)
        
        # Getting a list of zip files (each speaker is in a separate zip file)
        speaker_zips = [f for f in os.listdir(temp_dir) if f.endswith('.zip')]
        
        iteratable = []
        countforspeaker = 0
        
        for speaker_zip in speaker_zips:
            speaker_zip_path = os.path.join(temp_dir, speaker_zip)
            speaker_temp_dir = os.path.join(temp_dir, speaker_zip.replace('.zip', ''))
            # Extract the speaker's zip file
            with zipfile.ZipFile(speaker_zip_path, 'r') as speaker_zip_ref:
                speaker_zip_ref.extractall(speaker_temp_dir)
                
            internal_dir = os.listdir(speaker_temp_dir)[0]
            print(internal_dir)
            wav_dir = os.path.join(speaker_temp_dir, internal_dir, 'wav')
            annotation_dir = os.path.join(speaker_temp_dir, internal_dir, 'annotation')
            textgrid_dir = os.path.join(speaker_temp_dir, internal_dir, 'textgrid')
            transcript_dir = os.path.join(speaker_temp_dir, internal_dir, 'transcript')
            
            if os.path.exists(wav_dir): wav_files = get_all_files(wav_dir)
            if os.path.exists(annotation_dir): annotation_files = get_all_files(annotation_dir)
            if os.path.exists(textgrid_dir): textgrid_files = get_all_files(textgrid_dir)
            if os.path.exists(transcript_dir): transcript_files = get_all_files(transcript_dir)
            
            countforsample = 0
            for wav_file, transcript_file, textgrid_file, annotation_file in zip(wav_files, transcript_files, textgrid_files, annotation_files):
                words, tphones = extract_words_and_phones(textgrid_file)
                words, aphones = extract_words_and_phones(annotation_file)
                #list(map(lambda phone: phone[2], phones))
                print(words)
                print(tphones)
                print(aphones)
                
                ipa_transcription = convert_arpabet_to_ipa(tphones)
                
                sample_rate, data = wavfile.read(wav_file)
                with open(transcript_file, 'r') as file:
                    iteratable.append((data, ipa_transcription))
                    
                countforsample+=1
                if (countforsample == numberofsampleperSpeaker):
                    break
                    
            # for wav_file in wav_files:
            #     sample_rate, data = wavfile.read(wav_file)
            #     #print(data)
            #     audios.append(data)
            
            countforspeaker+=1
            if (numberofspeaker == countforspeaker):
                break
                
                
                
        
        shutil.rmtree(temp_dir)
        return iteratable
                
        
            

def get_all_files(directory):
    files_and_dirs = os.listdir(directory)
    
    files_paths = []
    files = [f for f in files_and_dirs if os.path.isfile(os.path.join(directory, f))]
    
    for file in files:
        files_paths.append(os.path.join(directory, file))
    
    return files_paths


def extract_words_and_phones(textgrid_path):
    # Load the TextGrid file
    tg = tgio.openTextgrid(textgrid_path, includeEmptyIntervals=False)

    # Assuming the names of the tiers are 'words' and 'phones'
    words_tier = tg.getTier('words')
    phones_tier = tg.getTier('phones')

    # Extract words and their intervals
    words = [(interval.start, interval.end, interval.label) for interval in words_tier.entries]

    # Extract phones and their intervals
    phones = [(interval.start, interval.end, interval.label) for interval in phones_tier.entries]

    return words, phones



def convert_arpabet_to_ipa(arpabet_phones):
    ipa_phones = [arpabet_to_ipa[phone[2]] for phone in arpabet_phones if phone[2] in arpabet_to_ipa]
    return ''.join(ipa_phones)



result = get_data_iterator()
print(result)


ABA
[(0.03, 0.33, 'for'), (0.39, 0.62, 'the'), (0.62, 1.24, 'twentieth'), (1.52, 1.91, 'time'), (2.22, 2.65, 'that'), (2.65, 3.14, 'evening'), (3.18, 3.46, 'the'), (3.46, 3.71, 'two'), (3.71, 4.07, 'men'), (4.12, 4.48, 'shook'), (4.48, 4.87, 'hands')]
[(0.0, 0.08, 'sil'), (0.08, 0.25, 'AO1'), (0.25, 0.41, 'TH'), (0.41, 0.59, 'ER0'), (0.59, 0.63, 'sp'), (0.63, 0.77, 'AH1'), (0.77, 0.87, 'V'), (0.87, 0.91, 'DH'), (0.91, 0.99, 'AH1'), (0.99, 1.06, 'D'), (1.06, 1.1, 'EY1'), (1.1, 1.15, 'N'), (1.15, 1.23, 'JH'), (1.23, 1.31, 'ER0'), (1.31, 1.44, 'T'), (1.44, 1.54, 'R'), (1.54, 1.63, 'EY1'), (1.63, 1.74, 'L'), (1.74, 1.84, 'sp'), (1.84, 1.94, 'F'), (1.94, 1.97, 'IH1'), (1.97, 2.0, 'L'), (2.0, 2.07, 'IH0'), (2.07, 2.13, 'P'), (2.13, 2.22, 'S'), (2.22, 2.31, 'T'), (2.31, 2.42, 'IY1'), (2.42, 2.49, 'L'), (2.49, 2.64, 'Z'), (2.64, 2.75, 'EH2'), (2.75, 2.83, 'T'), (2.83, 2.94, 'S'), (2.94, 2.97, 'EH1'), (2.97, 3.0, 'T'), (3.0, 3.03, 'ER0'), (3.03, 3.08, 'AH0'), (3.08, 3.18, 'sp')]
[(0.0, 0.03, 's

In [3]:
# generate predictions
def step_generate_pfer(prediction_frame: pl.DataFrame, 
                                  **kwargs):
    feat_edit_distance = panphon.distance.Distance().feature_edit_distance
    # currently, the predictions have spaces between segments. Remove the spaces
    # in the predictions column
    prediction_frame = prediction_frame.with_columns([
        pl.col('predictions').map_elements(lambda x: x.replace(' ', '')).alias('predictions')
    ])
    prediction_frame = prediction_frame.with_columns([
        pl.struct(['predictions', 'transcripts']).map_elements(
            lambda x: feat_edit_distance(x['predictions'], x['transcripts'])).alias('pfer')
    ])
    # group by the language code and compute the average pfer
    return prediction_frame

In [4]:
def step_generate_predictions_notre_dame(**kwargs) -> str:
    model = Wav2Vec2ForCTC.from_pretrained("ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns", cache_dir=HF_CACHE_DIR)
    processor = Wav2Vec2Processor.from_pretrained("ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns", cache_dir=HF_CACHE_DIR)
    predictions = []
    transcripts = []
    for datapoint in get_data_iterator(): # TODO: you have to implement this function to iterate over the arctic samples
        audio, txt = datapoint # TODO: the iterator should returns tuples of the audio array and the transcript
        #print(audio)
        input_values = processor(audio, return_tensors="pt", sampling_rate=16000).input_values
        # Convert input_values to double
        input_values_double = input_values.double()

        # Ensure the model is in double precision
        model = model.double()  
        with torch.no_grad():
            logits = model(input_values_double).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        prediction = processor.batch_decode(predicted_ids)[0]
        predictions.append(prediction)
        transcripts.append(txt)
        
    print(pl.DataFrame({
        "predictions": predictions,
        "transcripts": transcripts
    }))
    return pl.DataFrame({
        "predictions": predictions,
        "transcripts": transcripts
    })


In [5]:
if __name__ == "__main__":
    step_dict = OrderedDict()

    step_dict['step_generate_preds_notre_dame'] = SingletonStep(step_generate_predictions_notre_dame, {
        "version": "001"
    })
    
    # step_dict['step_generate_pfer_notre_dame'] = SingletonStep(step_generate_pfer, {
    #     'version': '001', 
    #     'prediction_frame': 'map_step_generate_notredame_preds'
    # })
    metadata = conduct(os.path.join(SCRATCH_DIR, "arctic_flowmason_cache"), step_dict, "arctic_experiment_logs")
    ipdb.set_trace()

2024-02-09 13:12:46.942 | INFO     | flowmason.dag:conduct:314 - Step step_generate_preds_notre_dame is cached at C:\Users\david\OneDrive\Desktop\school\URO\URO_multilingual_speech_recognition\outputs\scratch_dir\arctic_flowmason_cache\d048277d42c6db1a98a4ff070727e347eaf6fccfc96700e07a1641e531b464b4, continuing.


--Return--
None
> c:\users\david\appdata\local\temp\ipykernel_17564\2046869975.py(13)<module>()

